In [41]:
import random
import numpy as np
import cec2017.functions as functions
import copy

In [42]:
Dim = 10
NP = 100
bound = 100

In [43]:
X = np.random.uniform(low=-bound, high=bound, size=Dim)
X_min = X.min()
X_max = X.max()
X_list = np.empty([NP, Dim])
for i in range(NP):
    for j in range(Dim):
        X_list[i][j] = X_min + random.uniform(0, 1) * (X_max - X_min)

X_list

array([[ 4.83578534e+01,  6.18359847e+01,  8.35451953e+00,
        -2.47067325e+01, -8.76542098e+01, -7.79072048e+01,
        -6.72112443e+01, -2.45794055e+01, -6.93339965e+01,
        -2.32607910e+01],
       [-9.25632201e+01,  7.62613731e+01,  6.60672962e+01,
        -4.75057396e+01,  1.80797611e+01, -6.30336728e+01,
         6.23283463e+01, -1.52386136e+01, -9.30711449e+01,
        -5.87808251e+01],
       [ 4.78772600e+00, -6.52952157e+01, -6.96910046e+00,
         6.81321099e+01,  2.77411567e+01,  5.18658089e+01,
        -8.46186875e+01,  5.23764628e+01, -5.23493932e+01,
        -3.64829129e+01],
       [ 3.39546970e+00, -4.39248891e+01, -7.61983698e+01,
        -2.29250879e+01,  1.16208742e+01,  9.03469209e+01,
         2.41316990e+00,  3.05615153e+01, -7.91182022e+01,
        -8.55556318e+01],
       [-6.98924286e+01, -5.37316028e+01, -1.95334062e+01,
        -8.41394877e-01,  4.80914601e+01, -7.10700976e+01,
         5.78895082e+01,  3.46904876e+01, -5.17729645e+01,
         3.

In [44]:
""":parameter for DE"""
T_DE = 100
no = 5
CR = 0.9  # rate of crossover

""":parameter for MMS"""
mms_e = 0.000001
mms_F = 0.5

In [45]:
""":parameter k and F: for current/rand/1"""
k = 0.3
F = 0.5

In [46]:
""":parameter for crossoverSBX"""
sbx_n = 10

In [47]:
# f = functions.f1

In [48]:
def feas(x):
    if x.max() <= bound and x.min() >= -bound:
        return 1
    else:
        return 0

In [49]:
def select_better_fit(x1, x2, f):
    if f(x1) <= f(x2):
        return x1
    else:
        return x2

In [50]:
def current_to_rand(x_list, r1, r2, r3, i):
    return np.add(x_list[i], k * np.subtract(x_list[r1], x_list[i]), F * np.subtract(x_list[r2], x_list[r3]))

In [51]:
def immsade(xx_list, r1, r2, r3):
    immsade_omega = random.uniform(0.7, 1.0)
    immsade_F = random.uniform(0.5, 0.8)
    v_delta = np.add(np.multiply(immsade_omega, xx_list[r1]),
                     immsade_F * np.subtract(xx_list[r2], xx_list[r3]))
    v_delta = np.maximum(v_delta, -100)
    v_delta = np.minimum(v_delta, 100)
    return v_delta

In [52]:
def crossover_SBX(x_i, v_j, f):
    beta = 0
    while isinstance(beta, complex):
        sbx_rand = random.random()
        if sbx_rand <= 0.5:
            beta = (2 * sbx_rand) ** (1 / (1 + sbx_n))
        else:
            beta = (1 / (2 - sbx_n * 2)) ** (1 / (1 + sbx_n))

    c1 = 0.5 * np.add((1 + beta) * x_i, (1 - beta) * v_j)  # crossover
    c2 = 0.5 * np.add((1 - beta) * x_i, (1 + beta) * v_j)
    return select_better_fit(c1, c2, f)

In [53]:
def DE(f):
    x_list = copy.deepcopy(X_list)
    x_best = x_list[0]
    # print(x_list[0])

    t = 1
    while t <= T_DE:
        for i in range(NP):
            u_best = np.empty([Dim])
            for j in range(Dim):
                r_list = random.sample(range(NP), 5)
                for m in range(no):
                    while r_list[m] == i:
                        r_list[m] = random.randint(0, Dim - 1)

                v_j = np.empty([Dim])
                v_j = immsade(x_list, r_list[1], r_list[2], r_list[3])

                cr_rand = random.random()
                if cr_rand <= CR:
                    u_j = crossover_SBX(x_list[i], v_j, f)
                else:
                    u_j = x_list[i]

                if j == 0:
                    u_best = u_j
                else:
                    u_best = select_better_fit(u_best, u_j, f)

                if f(u_best) <= f(x_list[i]) and feas(u_best) == 1:
                    x_list[i] = u_best

                if f(x_list[i]) < f(x_best) and feas(x_list[i]) == 1:
                    x_best = x_list[i]

        t += 1
    return x_best


In [54]:
""":parameter: for AGDE"""
T_AGDE = 100

In [55]:
def index_of_X_best(xx_list, f):
    index = 0
    min_fit = f(xx_list[0])
    for i in range(len(xx_list)):
        if f(xx_list[i]) < min_fit:
            index = i
            min_fit = f(xx_list[i])
    return index


def index_of_X_worst(xx_list, f):
    index = 0
    max_fit = f(xx_list[0])
    for i in range(len(xx_list)):
        if f(xx_list[i]) > max_fit:
            index = i
            max_fit = f(xx_list[i])
    return index


def compare_max(delta):
    """
    lower bound
    :param delta: delta
    :return: turn into lower bound delta
    """

    em = np.linspace(-100, -100, Dim)
    delta = np.maximum(delta, em)
    return delta


def compare_min(delta):
    on = np.linspace(100, 100, Dim)
    delta = np.minimum(delta, on)
    # for i in range(len(delta)):
    #     delta[i] = min(delta[i], 1)
    return delta

def agde(xx_list, agde_F, agde_p, f):
    agde_NP = int(NP * agde_p)
    r_list = random.randint(agde_NP * 2 - 1, NP - 1)
    x_p_worst = xx_list[index_of_X_worst(xx_list[agde_NP: agde_NP * 2 - 1], f)]
    x_p_best = xx_list[index_of_X_best(xx_list[: agde_NP - 1], f)]
    v_delta = np.add(xx_list[r_list], np.multiply(agde_F, np.subtract(x_p_best, x_p_worst)))
    return v_delta

In [56]:
def AGDE(f):
    xx_list = copy.deepcopy(X_list)
    x_best = xx_list[0]
    # print(xx_list[0])

    G = 0
    p_j = 0.5
    s_j_sum = 0
    while G <= T_AGDE:  # iteration of DE
        n_s_j = 0
        f_j = 1
        for i in range(NP):  # iteration of generation
            agde_F = random.uniform(0.1, 1)
            Cr = 0
            if G == 1:
                if random.uniform(0, 1) <= 0.5:
                    Cr = random.uniform(0.05, 0.15)
                else:
                    Cr = random.uniform(0.9, 1)
            else:
                if random.uniform(0, 1) <= p_j:
                    Cr = random.uniform(0.05, 0.15)
                else:
                    Cr = random.uniform(0.9, 1)
            j_random = random.randint(1, NP)

            u_best = np.empty([Dim])
            for j in range(Dim):
                vj = agde(xx_list, agde_F, p_j, f)
                vj = compare_max(vj)
                vj = compare_min(vj)
                if random.uniform(0, 1) < Cr or j == j_random:
                    u_j = crossover_SBX(xx_list[i], vj, f)
                else:
                    u_j = xx_list[i]

                if j == 0:
                    u_best = u_j
                else:
                    u_best = select_better_fit(u_best, u_j, f)

                if f(u_best) <= f(xx_list[i]) and feas(u_best) == 1:
                    xx_list[i] = u_best

                if f(xx_list[i]) < f(x_best) and feas(xx_list[i]) == 1:
                    x_best = xx_list[i]

        G += 1

        s_j = n_s_j / (n_s_j + f_j) + 0.01
        s_j_sum += s_j
        p_s_j = s_j / s_j_sum
        if (((G - 1) * p_j + p_s_j) / G) < 0.5:
            p_j = ((G - 1) * p_j + p_s_j) / G

    return x_best

In [57]:
""":parameter: for GSK"""
T_GSK = 100
GSK_p = 0.1
k_f = 0.01   # 0.5
k_r = 0.1   # 0.9
K = 10

In [58]:
def junior(xx_list, r, i, f):
    if random.uniform(0, 1) <= k_r:
        if f(xx_list[i]) > f(xx_list[r]):
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(xx_list[max(0, i - 1)], xx_list[min(NP - 1, i + 1)]),
                                          np.subtract(xx_list[r], xx_list[i])))
        else:
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(xx_list[max(0, i - 1)], xx_list[min(NP - 1, i + 1)]),
                                          np.subtract(xx_list[i], xx_list[r])))
    else:
        v_delta = xx_list[i]
    return v_delta


def senior(xx_list, i, D_junior, f):
    if random.uniform(0, 1) <= k_r:
        D_senior = NP - D_junior
        NP_1 = int(D_senior * GSK_p)
        r_list = random.randint(D_junior - 1 + NP_1 * 2, NP - 1)
        x_p_worst = xx_list[index_of_X_worst(xx_list[D_junior - 1 + NP_1: D_junior - 1 + NP_1 * 2 - 1], f)]
        x_p_best = xx_list[index_of_X_best(xx_list[max(D_junior - 1, 0): D_junior - 1 + NP_1 - 1], f)]
        if f(xx_list[i]) > f(xx_list[r_list]):
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(x_p_best, x_p_worst),
                                          np.subtract(xx_list[r_list], xx_list[i])))
        else:
            v_delta = np.add(xx_list[i],
                             k_f * np.add(np.subtract(x_p_best, x_p_worst),
                                          np.subtract(xx_list[i], xx_list[r_list])))
    else:
        v_delta = xx_list[i]
    return v_delta

In [59]:
def GSK(f):
    """
    DE algorithm
    :param x_list:
    :return: best x by DE
    """
    xxx_list = copy.deepcopy(X_list)

    x_best = xxx_list[0]
    # print(xxx_list[0])
    G = 0

    while G <= T_GSK:  # iteration of DE
        D_junior = int(NP * pow((1 - G / T_GSK), K))

        for i in range(NP):  # iteration of generation

            u_best = np.empty([Dim])
            for j in range(Dim):
                vj = np.empty([Dim])
                r = random.randint(0, NP - 1)
                if j <= D_junior:
                    vj = junior(xxx_list, r, j, f)
                else:
                    vj = senior(xxx_list, j, D_junior, f)

                vj = compare_max(vj)
                vj = compare_min(vj)

                if j == 0:
                    u_best = vj
                else:
                    u_best = select_better_fit(u_best, vj, f)

                if f(u_best) <= f(xxx_list[i]) and feas(u_best) == 1:
                    xxx_list[i] = u_best

                if f(xxx_list[i]) < f(x_best) and feas(xxx_list[i]) == 1:
                    x_best = xxx_list[i]

        G += 1

    return x_best


In [ ]:
import os
i = 0
test_list = np.empty([len(functions.all_functions), 3])
test_path = r'D:\profile\codes\python_projects\ocp\test\output\D10_cpu'
for i in range(51):
    for f in functions.all_functions:
        x_best_DE_fit = 0
        x_best_AGDE_fit = 0
        x_best_GSK_fit = 0
        x_best_DE_fit = f(DE(f))
        x_best_AGDE_fit = f(AGDE(f))
        x_best_GSK_fit = f(GSK(f))
        print(f.__name__, x_best_DE_fit, x_best_AGDE_fit, x_best_GSK_fit, "\n")
        test_list[i][0] = x_best_DE_fit
        test_list[i][1] = x_best_AGDE_fit
        test_list[i][2] = x_best_GSK_fit

    i += 1
    np.savetxt(os.path.join(test_path, str(i) + "\n"), test_list)




f1 7947413255.988082 48338595158.25612 117005597883.60776 

f2 8794426129.035284 55871146022.738625 22686972218485.832 

f3 7531.236152198845 17052.309288042008 46742.98206169039 

f4 756.5684818122801 784.9934972457892 2253.086521921467 

f5 563.9346742193015 550.9273859541177 649.8318426771716 

f6 623.4630558766038 627.6945382864205 674.0552356360415 

f7 758.5670050575397 773.07656349306 1077.8335466066687 

f8 820.2095318772191 852.0450511276558 914.9904935969496 

f9 940.3942752225055 1255.2443096579652 4069.2462569039817 

f10 2098.574265172232 2153.245227610032 2852.1205565038513 

f11 1295.5491192030702 1511.475359188936 4822.982194686475 

f12 73630584.96830912 492540.24593766266 11651501038.694456 

f13 282047.9091681239 6952.982114504324 389023680.54088575 

f14 3547.6891921587962 1665.3496858212893 2254.6100977588103 

f15 11118.77387021492 3126.5412997188814 43192893.87797752 

f16 1888.093094807298 1895.95892176796 2870.054809647944 



c:\users\chris\appdata\local\programs\python\python39\lib\site-packages\cec2017\basic.py:227: RuntimeWarning: invalid value encountered in true_divide
  t = np.abs(tjx - np.round(tjx)) / tj


f17 nan nan nan 

f18 18885.62595180027 2182.2107671765 279006478.6880716 

f19 95703.36344281424 2656.5585960926874 805028.1436041197 

f20 nan nan nan 

f21 2228.758124816006 2348.852881267853 2417.7034797285755 

f22 2439.0421844385446 2655.8981952739723 3286.5502296925197 

f23 2684.2984570645235 2683.056618361209 2760.373879484634 

f24 2711.664272881615 2902.545075329448 2963.0010239830463 

f25 3009.7667343453513 3017.085657137505 3482.022837827339 

f26 3116.042673043014 3758.659063544741 5373.971672638523 

f27 3132.829365064455 3231.5692193267896 3303.5627236661885 

f28 3355.9213117318814 3703.6245827666185 3921.573593685579 

f29 nan nan nan 

f30 9792134.394096052 11913686.125437822 76470188.65322202 

f1 17852548991.82711 38983878408.46362 117005597883.60776 

f2 202121755.3358657 8962379459.781532 22686972218485.832 

f3 5150.863831806227 16299.399589477229 46742.98206169039 

f4 580.9140080954112 889.6948140797633 2253.086521921467 

f5 550.4920667081648 554.08887726833

In [ ]:
import cec2017.utils as utils
for f in functions.all_functions:
    utils.surface_plot(f, points=120, dimension=Dim)